# Dataset - Telco Customer Churn

O dataset Telco Customer Churn, disponibilizado pela IBM, contém informações sobre clientes de uma empresa de telecomunicações e é utilizado para a análise e previsão do fenômeno de churn, ou seja, o cancelamento de serviços pelos clientes. Composto por 7.043 instâncias e 20 atributos, este conjunto de dados inclui tanto variáveis categóricas quanto numéricas, englobando aspectos como dados demográficos, informações contratuais, uso de serviços e valores monetários. A variável-alvo, Churn, classifica os clientes como aqueles que cancelaram ou não os serviços da empresa.

Entre as variáveis presentes no dataset, destacam-se informações como gender, SeniorCitizen, tenure, Contract, PaymentMethod, MonthlyCharges, e TotalCharges, que fornecem uma visão detalhada dos clientes e do relacionamento contratual com a empresa. No entanto, algumas colunas, como a TotalCharges, exigem pré-processamento adicional devido a inconsistências no tipo de dado.

Este dataset é amplamente utilizado para a construção de modelos preditivos de classificação binária, permitindo a análise de fatores que influenciam o churn dos clientes e contribuindo para a otimização de estratégias empresariais no setor de telecomunicações.

Features

1. gender
→ Gênero do cliente (masculino ou feminino).

2. SeniorCitizen
→ Se o cliente é idoso (1 = Sim, 0 = Não).

3. Partner
→ Se o cliente tem parceiro/parceira (Sim ou Não).

4. Dependents
→ Se o cliente tem dependentes (filhos, outros familiares).

5. tenure
→ Tempo de permanência na empresa (em meses).

6. PhoneService
→ Se o cliente possui serviço de telefone (Sim ou Não).

7. MultipleLines
→ Se o cliente possui múltiplas linhas telefônicas (Sim, Não ou Não possui serviço telefônico).

8. InternetService
→ Tipo de serviço de internet (DSL, fibra ótica ou nenhum).

9. OnlineSecurity
→ Se o cliente possui segurança online (proteção contra vírus).

10. OnlineBackup
→ Se o cliente possui serviço de backup online.

11. DeviceProtection
→ Se o cliente possui proteção para seus dispositivos (manutenção, suporte).

12. TechSupport
→ Se o cliente possui suporte técnico.

13. StreamingTV
→ Se o cliente possui serviço de streaming de TV.

14. StreamingMovies
→ Se o cliente possui serviço de streaming de filmes.

15. Contract
→ Tipo de contrato do cliente (mensal, anual, bianual).

16. PaperlessBilling
→ Se o cliente recebe a fatura apenas online (sem papel).

17. PaymentMethod
→ Método de pagamento (boleto eletrônico, cheque enviado, débito automático, cartão de crédito automático).

18. MonthlyCharges
→ Valor da fatura mensal do cliente (em dólares).

19. TotalCharges
→ Valor total pago pelo cliente até o momento (em dólares).

20. Churn
→ Se o cliente cancelou o serviço (Sim = cancelou, Não = ainda está ativo).

A análise exploratória de dados foi conduzida de acordo com a natureza e a estrutura dos dados apresentados. Cada tipo de dataframe, necessita de uma abordagem específica, considerando suas características.

# 1. Importação dos dados

## 1.1. Importação

In [ ]:
from google.colab import files

# Abre a janela para selecionar o arquivo
uploaded = files.upload()

# Depois, para ler o arquivo
import pandas as pd

matriz = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

## 1.2. Características do dataset bruto

In [ ]:
# Informações básicas
print(f"Número de instâncias: {matriz.shape[0]}")
print(f"Número de colunas: {matriz.shape[1]}\n")

print("Colunas e tipos de variáveis:")
print("-" * 50)

# Percentual para definir "poucas categorias"
limite_categoria = 0.10 * matriz.shape[0]  # 10% do número de instâncias

# Para cada coluna
for coluna in matriz.columns:
    tipo = matriz[coluna].dtype

    if pd.api.types.is_numeric_dtype(matriz[coluna]):
        print(f"🔹 {coluna}: Numérica")

    elif pd.api.types.is_object_dtype(matriz[coluna]) or pd.api.types.is_categorical_dtype(matriz[coluna]):
        n_categorias = matriz[coluna].nunique()
        print(f"🔸 {coluna}: Categórica ({n_categorias} categorias)")

        # Se o número de categorias for pequeno (menos de 10% do total de instâncias)
        if n_categorias <= limite_categoria:
            categorias = matriz[coluna].unique()
            print(f"    Categorias: {categorias}")

    else:
        print(f"🔹 {coluna}: Tipo especial ({tipo})")

print("-" * 50)

## 1.3. Visualização da matriz

In [ ]:
# Visualização da matriz
matriz.head()

# 2. Entendimento inicial do dataset

## 2.1. Removendo coluna de customerID

A coluna 'customerID' deve ser removida da análise porque ela funciona apenas como um identificador único para cada cliente e não carrega nenhuma informação útil para os modelos de machine learning. Como possui 7043 categorias distintas (uma para cada instância), ela não contribui para padrões ou relações entre as variáveis e pode até prejudicar o desempenho do modelo ao introduzir ruído ou complexidade desnecessária. Mantê-la pode levar o modelo a aprender associações espúrias que não se repetem em novos dados.

In [ ]:
#Remover primeira coluna
matriz = matriz.drop(columns=['customerID'])

## 2.2. Atualização dos nomes das colunas

Visualização dos nomes da colunas atuais

In [ ]:
# Nome das colunas
matriz.columns

## 2.3. Informações numéricas sobre a matriz

Informações:
* Número total de entradas (linhas)
* Número de colunas
* Nome de cada coluna
* Quantidade de valores não nulos em cada coluna
* Tipo de dado de cada coluna (int64, float64, object, etc.)
* Uso de memória aproximado

In [ ]:
matriz.info()

## 2.4. Quantidade de valores nulos

Exibir a quantidade de valores nulos por cada coluna da matriz

In [ ]:
# Quantidade de valores nulos.
matriz.isnull().sum()

## 2.5. Presença de valores inválidos

Nesta subseção são feitas as seguintes tarefas:
* A presença de "NaN" e "?" por coluna
* Identificação de valores suspeitos (espaços em branco)
* Remoção das linhas com valores ausentes
* Identificação das colunas que parecem ser numéricas, mas estão como objeto (texto)
* Conversão das colunas "object" para "float"

### 2.5.1 Presença de "NaN" e "?"

In [ ]:
# Para cada coluna
  # Coluna: nome
  # Tipo: categórica, binária, numérica inteira, etc.
  # Se possui valores NaN
  # Se possui o caractere "?" (muito usado como marcador de valor ausente)

# Função para analisar coluna
def analisar_coluna(col):
    serie = matriz[col]
    tipo_dado = serie.dtype
    valores_unicos = serie.dropna().unique()
    num_unicos = len(valores_unicos)

    # Detecta se há '?'
    possui_interrogacao = serie.astype(str).str.contains(r'^\?$').any()

    # Define tipo da variável
    if tipo_dado == 'object':
        if num_unicos == 2:
            tipo_variavel = 'Binária (categórica)'
        else:
            tipo_variavel = 'Categórica'
        escala = f"Categorias: {list(valores_unicos)}"
    elif pd.api.types.is_numeric_dtype(serie):
        if num_unicos == 2:
            tipo_variavel = 'Binária (numérica)'
        elif pd.api.types.is_integer_dtype(serie):
            tipo_variavel = 'Numérica inteira'
        else:
            tipo_variavel = 'Numérica contínua'
        escala = f"Valores entre {serie.min()} e {serie.max()}"
    else:
        tipo_variavel = 'Outro'
        escala = 'Não identificado'

    possui_nan = serie.isna().any()

    return {
        'Tipo': tipo_variavel,
        'Escala': escala,
        'Possui NaN': possui_nan,
        'Possui "?"': possui_interrogacao
    }

# Analisa todas as colunas
analise_completa = {col: analisar_coluna(col) for col in matriz.columns}

# Exibe o resultado de forma organizada
for col, info in analise_completa.items():
    print(f"\nColuna: {col}")
    for chave, valor in info.items():
        print(f"  {chave}: {valor}")

### 2.5.2. Identificar valores suspeitos: espaços em branco

In [ ]:
# Identificar valores suspeitos: espaços em branco
print("\nAnalisando valores suspeitos (espaços em branco ou strings vazias):")
for coluna in matriz.columns:
    if matriz[coluna].dtype == 'object':
        qtde_brancos = (matriz[coluna].str.strip() == '').sum()
        if qtde_brancos > 0:
            print(f"- {coluna}: {qtde_brancos} valores em branco")

### 2.5.3. Remoção dos espaços vazios

In [ ]:
# Remover linhas com valores ausentes (NaN) - string
matriz = matriz[matriz['TotalCharges'] != '']

# Remover linhas com valores ausentes (NaN) na coluna 'TotalCharges'
matriz = matriz.dropna(subset=['TotalCharges'])

# Verificar se as linhas com NaN foram removidas
print("\nLinhas com valores ausentes em 'TotalCharges' foram removidas:")
print(matriz.isnull().sum())


### 2.5.4. Colunas que parecem ser numéricas

In [ ]:
import pandas as pd

# Verificar colunas que estão como texto, mas deveriam ser numéricas
possiveis_numericas = []

for coluna in matriz.columns:
    if matriz[coluna].dtype == 'object':
        # Limpa espaços e remove strings vazias
        serie_limpa = matriz[coluna].dropna().astype(str).str.strip()
        serie_filtrada = serie_limpa[serie_limpa != '']
        try:
            pd.to_numeric(serie_filtrada, errors='raise')
            possiveis_numericas.append(coluna)
        except Exception as e:
            print(f"Falha ao converter '{coluna}': {e}")

if possiveis_numericas:
    print("\nAs seguintes colunas estão como 'object', mas parecem ser numéricas:")
    for col in possiveis_numericas:
        print(f"- {col}")
else:
    print("\nNão foram encontradas colunas com suspeita de tipo errado.")


### 2.5.5. Conversão da coluna "object" para "float"

A variável 'TotalCharges' deveria ser numérica, mas parece estar lida como texto/categórica.
Aqui, é feita a conversão da coluna para float.

In [ ]:
# Corrigir a variável TotalCharges
import pandas as pd
matriz['TotalCharges'] = pd.to_numeric(matriz['TotalCharges'], errors='coerce')

# Verifica o tipo de dados da coluna
print(matriz['TotalCharges'].dtype)

## 2.6. Verificação de colunas duplicadas

Esse código verifica se existem colunas duplicadas em um DataFrame — ou seja, colunas diferentes que têm exatamente os mesmos valores linha por linha. Ele faz isso comparando cada par de colunas entre si.

In [ ]:
# Verificar colunas duplicadas
duplicadas = []
colunas = matriz.columns

for i in range(len(colunas)):
    for j in range(i + 1, len(colunas)):
        if matriz[colunas[i]].equals(matriz[colunas[j]]):
            duplicadas.append((colunas[i], colunas[j]))

# Exibe os pares de colunas duplicadas
for c1, c2 in duplicadas:
    print(f"Colunas '{c1}' e '{c2}' são duplicadas.")

## 2.7. Verificação de valores únicos

Esse código verifica quantos valores únicos existem em cada coluna do DataFrame matriz. Serve para:
* Identificar colunas constantes (com apenas 1 valor).
* Detectar colunas com alta cardinalidade (ex: IDs, nomes únicos).
* Avaliar colunas que podem precisar de tratamento especial (como transformação categórica, remoção ou codificação).

In [ ]:
# Verificar valores únicos
valores_unicos = matriz.nunique()
print(valores_unicos)

## 2.8. Verificar linhas duplicadas

Identificar linhas que estão duplicadas e remove-as mantendo a primeira.

In [ ]:
#Detectar duplicatas - linhas
#Verifica se existem linhas repetidas no DataFrame:
duplicatas = matriz.duplicated()
print(matriz[duplicatas])

# Contar duplicatas - linhas
print(f"\nTotal de linhas duplicadas: {matriz.duplicated().sum()}")

# Remover duplicatas
# Remove as duplicatas mantendo a primeira ocorrência
matriz = matriz.drop_duplicates()

## 2.9. Verificar cardinalidade

A cardinalidade ajudar a identificar colunas que:
* São fáceis de codificar (baixa cardinalidade — poucos valores únicos, como "Sim"/"Não").
* Podem causar problemas (alta cardinalidade — muitos valores únicos, como "customerID", nomes ou códigos).
* Alta cardinalidade pode prejudicar modelos de machine learning, especialmente com codificações como one-hot (gera muitas colunas e risco de overfitting).
* Pode indicar colunas que devem ser removidas (como IDs).
* Ajuda na escolha da estratégia de pré-processamento (label encoding, hashing, etc.).

In [ ]:
# Verificar cardinalidade
# Seleciona apenas colunas categóricas (strings/objetos)
colunas_categoricas = matriz.select_dtypes(include='object')

# Total de linhas no DataFrame
total_linhas = len(matriz)

print("Cardinalidade das variáveis categóricas:")
print()

# Para cada coluna categórica, calcula e avalia a cardinalidade
# Mais de 10% das linhas têm categorias diferentes é considerada alta cardinalidade
for coluna in colunas_categoricas.columns:
    unicos = matriz[coluna].nunique()
    proporcao = unicos / total_linhas
    status = "ALTA" if proporcao > 0.1 else "BAIXA"

    print(f"Coluna: {coluna}")
    print(f"  → Valores únicos: {unicos}")
    print(f"  → Proporção: {proporcao:.2%}")
    print(f"  → Cardinalidade: {status}")
    print()

# 3. Estatísticas descritivas

## 3.1. Variáveis numéricas - descrição

Nesta subseção são analisadas as variáveis numéricas quanto a:
* Quantidade de valores não nulos
* Média
* Desvio padrão
* Quartos e valores extremos

In [ ]:
matriz.describe()

## 3.2. Variáveis categóricas - descrição e conversão

Nesta subseção são analisadas as variáveis categóricas quanto a:
* Descrição: frequência da quantidade de valores em cada coluna.
* Label Encoding: conversão e mapeamento das variáveis binárias para valores numéricos
* Verificação se existem valores inesperados após a conversão Label Encoding.
* One-Hot Encoding: conversão e mapeamento das variáveis com mais de 2 categoria. Bem como, criação de novas colunas, remoção de colunas redundantes E descrição das colunas finais.
* Verificação se existem valores inesperados após a conversão One-Hot Encoding.
* Verifica a presença de valores "NaN" e "?" após a codificação One-Hot Endoding
* Remove linhas com valores "NaN"
* Visualiza a matriz após transformações

### 2.2.1. Descrição

In [ ]:
# DESCRIÇÃO - VARIÁVEIS CATEGÓRICAS
  # Os valores únicos em cada coluna categórica
  # Quantas vezes cada um aparece (frequência absoluta)

# Seleciona colunas categóricas
colunas_cat = matriz.select_dtypes(include='object')

# Frequência de valores em cada coluna categórica
print("\nFrequência dos valores nas variáveis categóricas:")
for col in colunas_cat.columns:
    print(f"\nColuna: {col}")
    print(matriz[col].value_counts())

### 2.2.2. Variáveis binárias - Label Encoding

In [ ]:
import pandas as pd

# Colunas binárias
colunas_binarias = [
    'gender', 'Partner', 'Dependents', 'PhoneService',
    'PaperlessBilling', 'Churn'
]

# Diagnóstico: valores únicos antes do mapeamento
print("Valores únicos antes do mapeamento:")
print("--------------------------------------")
for coluna in colunas_binarias:
    print(f"- {coluna}: {matriz[coluna].unique()}")


# Mapeamento binário
for coluna in colunas_binarias:
    if coluna == 'gender':
        matriz[coluna] = matriz[coluna].map({'Female': 0, 'Male': 1})
    else:
        matriz[coluna] = matriz[coluna].map({'No': 0, 'Yes': 1})

# Verificação final: valores únicos e possíveis NaNs
print("\nMapeamento realizado nas variáveis binárias:")
print("--------------------------------------")
print("- gender: Female = 0, Male = 1")
print("- Partner: No = 0, Yes = 1")
print("- Dependents: No = 0, Yes = 1")
print("- PhoneService: No = 0, Yes = 1")
print("- PaperlessBilling: No = 0, Yes = 1")
print("- Churn: No = 0, Yes = 1")

print("\nValores únicos após o mapeamento:")
print("--------------------------------------")
for coluna in colunas_binarias:
    print(f"- {coluna}: {matriz[coluna].unique()}")

# Checar se há NaN
print("\nVerificando se há valores NaN após o mapeamento:")
print(matriz[colunas_binarias].isnull().sum())


### 2.2.3. Verificar a presença de valores inesperados após a conversão - Label Encoding

In [ ]:
#Verificar se existem valores inesperados:
for coluna in colunas_binarias:
    print(f"{coluna}: {matriz[coluna].unique()}")

### 2.2.4. Variáveis com mais de 2 categorias - One-Hot Encoding

In [ ]:
# Colunas com 3 ou mais categorias (One-Hot Encoding)
colunas_multicategoricas = [
    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
    'Contract', 'PaymentMethod'
]

# Diagnóstico: categorias únicas antes do One-Hot Encoding
print("Categorias únicas antes do One-Hot Encoding:")
print("-------------------------------------------------")
categorias_removidas = []

for coluna in colunas_multicategoricas:
    categorias = matriz[coluna].dropna().unique().tolist()
    print(f"- {coluna}: {categorias}")
    if len(categorias) > 1:
        categorias_removidas.append(f"{coluna}_{categorias[0]}")  # Primeira será descartada com drop_first

# Aplicar One-Hot Encoding com drop_first=True
matriz = pd.get_dummies(matriz, columns=colunas_multicategoricas, drop_first=True)

# Identificar novas colunas criadas
novas_colunas = [col for col in matriz.columns if any(col.startswith(prefix + "_") for prefix in colunas_multicategoricas)]

# Converter booleanos para inteiros (0 e 1)
matriz[novas_colunas] = matriz[novas_colunas].astype(int)

# Detectar e remover colunas redundantes (ex: "No internet service", "No phone service")
redundantes = [col for col in novas_colunas if any(txt in col.lower() for txt in ['no internet service', 'no phone service'])]

matriz.drop(columns=redundantes, inplace=True)

# Exibir resumo das variáveis descartadas
print("\nVariáveis removidas com drop_first (1ª categoria de cada variável):")
print("----------------------------------------------------------------------")
print(f"Total removidas (drop_first): {len(categorias_removidas)}")
for cat in categorias_removidas:
    print(f"- {cat}")

print("\nVariáveis removidas por redundância lógica (ex: 'No internet service'):")
print("------------------------------------------------------------------------")
print(f"Total removidas (redundantes): {len(redundantes)}")
for col in redundantes:
    print(f"- {col}")

# Exibir as novas colunas finais após limpeza
print("\nColunas finais após One-Hot Encoding e limpeza:")
print("------------------------------------------------")
for coluna in colunas_multicategoricas:
    relacionadas = [c for c in matriz.columns if c.startswith(coluna + "_")]
    print(f"- {coluna}: {relacionadas}")


### 2.2.5.  Verificar a presença de valores inesperados após a conversão - One-Hot Encoding

In [ ]:
# Verificar se existem valores inesperados nas colunas
print("\nVerificando valores únicos nas colunas criadas pelo One-Hot Encoding:")
print("-------------------------------------------------------------------------")
for coluna in colunas_multicategoricas:
    colunas_oh = [c for c in matriz.columns if c.startswith(coluna + "_")]
    for c in colunas_oh:
        valores_unicos = matriz[c].unique()
        print(f"{c}: {valores_unicos}")

MultipleLines_No: Se o valor é True, significa que o cliente não tem múltiplas linhas, e o valor 1 será atribuído. Se o valor for False, o cliente tem múltiplas linhas, e o valor será 0.

### 2.2.6. Verifica a presença de valores "NaN" e "?" após a codificação One-Hot Encoding

In [ ]:
# Para cada coluna
  # Coluna: nome
  # Tipo: categórica, binária, numérica inteira, etc.
  # Se possui valores NaN
  # Se possui o caractere "?" (muito usado como marcador de valor ausente)

# Função para analisar colunas com NaN ou '?'
def colunas_com_nan_ou_interrogacao(df):
    colunas_problema = []

    for col in df.columns:
        serie = df[col]
        tem_nan = serie.isna().sum()
        tem_interrogacao = (serie.astype(str) == '?').sum()

        if tem_nan > 0 or tem_interrogacao > 0:
            colunas_problema.append({
                'Coluna': col,
                'Qtd_NaN': tem_nan,
                'Qtd_Interrogacao': tem_interrogacao
            })

    return colunas_problema

# Aplicar função e exibir resultados
problemas = colunas_com_nan_ou_interrogacao(matriz)

print("\nColunas com valores ausentes (NaN) ou '?' detectadas:")
print("--------------------------------------------------------")
for item in problemas:
    print(f"- {item['Coluna']}: {item['Qtd_NaN']} NaN, {item['Qtd_Interrogacao']} '?'")


### 2.2.7. Remove linhas com valores "NaN"

In [ ]:
# Verificar quantidade total de NaNs antes
qtd_nan_antes = matriz.isna().sum().sum()
linhas_antes = matriz.shape[0]

print(f"Total de valores NaN antes da remoção: {qtd_nan_antes}")
print(f"Número de linhas antes da remoção: {linhas_antes}")

# Remover todas as linhas com qualquer NaN
matriz = matriz.dropna()

# Verificar quantidade total de NaNs depois
qtd_nan_depois = matriz.isna().sum().sum()
linhas_depois = matriz.shape[0]

print("\nRemoção de linhas com valores NaN concluída.")
print(f"Número de linhas após a remoção: {linhas_depois}")
print(f"Total de valores NaN após a remoção: {qtd_nan_depois}")
print(f"Total de linhas removidas: {linhas_antes - linhas_depois}")

### 2.2.7. Visualiza a matriz após transformações

In [ ]:
matriz.head()

## 3.3. Medidas da estatística descritiva - após codificação das colunas

Aqui, é feita uma análise da estatística descritiva das variáveis numéricas:
* Média, Mediana, Moda, Desvio Padrão, Variância, IQR, Assimetria, Curtose.

Análise feita após a codificação das colunas antes categóricas e agora numéricas.

In [ ]:
# Para variáveis numéricas
# Separa numéricas e categóricas
num_df = matriz.select_dtypes(include='number')
cat_df = matriz.select_dtypes(include='object')

# Estatísticas para variáveis numéricas
media = num_df.mean()
mediana = num_df.median()
moda_num = num_df.mode().iloc[0]
desvio = num_df.std()
variancia = num_df.var()
q1 = num_df.quantile(0.25)
q3 = num_df.quantile(0.75)
iqr = q3 - q1
skew = num_df.skew()
kurt = num_df.kurt()

# Monta DataFrame com numéricas
estatisticas_numericas = pd.DataFrame({
    'Média': media,
    'Mediana': mediana,
    'Moda': moda_num,
    'Desvio Padrão': desvio,
    'Variância': variancia,
    'IQR (Q3 - Q1)': iqr,
    'Assimetria (Skewness)': skew,
    'Curtose': kurt
})

# Exibe com 3 casas decimais nas numéricas
pd.options.display.float_format = '{:.3f}'.format
print(estatisticas_numericas)

# 4. Visualizações Univariadas

3 tipos de visualizações univariadas são adequadas para esse contexto:
* Histograma: permite visualizar a distribuição de frequências de uma variável numérica contínua, como o tempo de permanência (tenure) ou o valor cobrado mensalmente (MonthlyCharges), facilitando a identificação de padrões, concentrações e assimetrias nos dados.
* Boxplot: apresenta a mediana, os quartis e os outliers de uma variável, sendo útil para entender a dispersão e possíveis valores extremos, como em TotalCharges, além de permitir comparações entre grupos, como clientes que cancelaram ou não (Churn).
* Densidade/KDE: fornece uma estimativa suave da distribuição de uma variável contínua, servindo como alternativa ao histograma, e é especialmente útil para comparar visualmente diferentes subgrupos dentro de uma mesma variável.

## 4.1. Histograma - variáveis numéricas

In [ ]:
# HISTOGRAMA - variáveis numéricas
import matplotlib.pyplot as plt

for col in matriz.select_dtypes(include='number').columns:
    plt.figure(figsize=(6,4))
    matriz[col].hist(bins=30)
    plt.title(f'Histograma - {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.grid(True)
    plt.show()


## 4.2. Boxplots - variáveis numéricas

In [ ]:
# BOXPLOTS - variáveis numéricas
import seaborn as sns
import matplotlib.pyplot as plt

# Filtrar apenas variáveis numéricas com mais de 2 valores distintos
colunas_multivalor = [
    col for col in matriz.select_dtypes(include='number').columns
    if matriz[col].nunique() > 2
]

# Gerar boxplots
for col in colunas_multivalor:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=matriz[col])
    plt.title(f'Boxplot - {col}')
    plt.xlabel(col)
    plt.show()


## 4.3. Densidade/KDE (Kernel Density Estimation)

In [ ]:
# Densidade / KDE (Kernel Density Estimation)
import seaborn as sns
import matplotlib.pyplot as plt

# Loop apenas nas variáveis numéricas com mais de 2 valores únicos
for col in matriz.select_dtypes(include='number').columns:
    if matriz[col].nunique() > 2:
        plt.figure(figsize=(6, 4))
        sns.kdeplot(data=matriz, x=col, fill=True)
        plt.title(f'Densidade (KDE) - {col}')
        plt.xlabel(col)
        plt.ylabel('Densidade')
        plt.grid(True)
        plt.show()


# 5. Visualizações Bivariadas

2 tipos de visualizações bivariadas são adequadas para esse contexto:

* Gráfico de dispersão (scatterplot): permite visualizar a relação entre duas variáveis numéricas, como a correlação entre MonthlyCharges e TotalCharges. Ele ajuda a identificar padrões, tendências lineares ou não lineares e a presença de outliers entre as variáveis.
* Mapas de calor de correlação (heatmap): mostra a intensidade da correlação entre várias variáveis, geralmente com uma matriz de correlação. Esse tipo de visualização é útil para entender rapidamente quais variáveis possuem uma relação forte ou fraca entre si, como por exemplo, a correlação entre tenure, MonthlyCharges e TotalCharges.

## 5.1. Gráfico de dispersão (scatterplot)

In [ ]:
 # Gráfico de Dispersão (Scatterplot) - Relação entre duas variáveis numéricas
import seaborn as sns
import matplotlib.pyplot as plt

# Selecionar variáveis numéricas com mais de 2 valores únicos
variaveis_validas = [
    col for col in matriz.select_dtypes(include='number').columns
    if matriz[col].nunique() > 2
]

# Gerar scatterplots entre combinações dessas variáveis
for i in range(len(variaveis_validas)):
    for j in range(i + 1, len(variaveis_validas)):
        plt.figure(figsize=(6,4))
        sns.scatterplot(data=matriz, x=variaveis_validas[i], y=variaveis_validas[j])
        plt.title(f'Scatterplot: {variaveis_validas[i]} vs {variaveis_validas[j]}')
        plt.tight_layout()
        plt.show()

## 5.2. Mapas de calor de correlação (heatmap)

In [ ]:
#Mapa de Calor de Correlação (Heatmap): correlações entre variáveis numéricas.
correlacao = matriz.corr()

plt.figure(figsize=(20, 14))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', fmt=".2f", square=True)
plt.title('Mapa de Correlação')
plt.show()

# 6. Detecção e remoção de Outliers

## 6.1. Avaliando a distribuição de cada variável

O código tem como objetivo avaliar a distribuição de cada variável numérica no DataFrame, para determinar se elas seguem uma distribuição normal ou se apresentam assimetrias.

In [ ]:
# Avaliando a distribuição de cada variável
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import pandas as pd

# Selecionar variáveis numéricas
variaveis_numericas = matriz.select_dtypes(include='number')

# DataFrame para armazenar os resultados
resultado_distribuicao = pd.DataFrame(columns=['Variável', 'Skewness', 'p-valor', 'Tipo de Distribuição'])

# Loop pelas variáveis
for col in variaveis_numericas.columns:
    # Plot
    sns.displot(matriz[col], kde=True)
    plt.title(f'Distribuição de {col}')
    plt.xlabel(col)
    plt.ylabel('Frequência')
    plt.show()

    # Cálculo de skewness
    skew = matriz[col].skew()

    # Teste de normalidade (Shapiro-Wilk, adequado para até 5000 amostras)
    # Reduz o tamanho se houver muitos dados
    amostra = matriz[col].dropna()
    if len(amostra) > 5000:
        amostra = amostra.sample(5000, random_state=42)
    stat, p = shapiro(amostra)

    # Classificação da distribuição
    if p > 0.05 and abs(skew) < 0.5:
        tipo = 'Aproximadamente normal'
    elif skew > 0.5:
        tipo = 'Assimétrica à direita'
    elif skew < -0.5:
        tipo = 'Assimétrica à esquerda'
    else:
        tipo = 'Levemente assimétrica'

    # Adiciona ao DataFrame de resultados
    resultado_distribuicao.loc[len(resultado_distribuicao)] = [col, round(skew, 2), round(p, 4), tipo]

# Exibe a tabela final
print("\nResumo das distribuições:")
print(resultado_distribuicao)


## 6.2. IQR

IQR (Intervalo Interquartil): Mede a dispersão entre o 25º e o 75º percentil.

* Detecção de outliers: Outliers podem distorcer análises estatísticas e a performance de modelos de aprendizado de máquina. Identificar e tratar esses valores é importante, especialmente em modelos que são sensíveis a dados extremos.

* Definição do multiplicador: O multiplicador (3, neste caso) é um fator que determina a quantidade de "força" que você permite para um valor se desviar da média. Quando o multiplicador é maior (por exemplo, 3), você é mais rigoroso ao identificar outliers.

* Tratamento de outliers: Após identificar os outliers, você pode decidir se deseja removê-los, transformá-los ou tratá-los de outra forma, dependendo da natureza dos dados e da análise.

In [ ]:
#IQR (Intervalo Interquartil): Mede a dispersão entre o 25º e o 75º percentil. Valores fora de 3 vezes o IQR são considerados outliers.
# Calcular IQR
# IQR (Intervalo Interquartil): Detecta outliers para qualquer variável numérica
import pandas as pd

# Seleciona apenas colunas numéricas
variaveis_numericas = matriz.select_dtypes(include='number')

# Lista para armazenar índices de outliers e contagem por coluna
outlier_indices = set()
contagem_por_coluna = {}
multiplicador = 3
# Loop para cada variável numérica
for coluna in variaveis_numericas.columns:
    Q1 = matriz[coluna].quantile(0.25)
    Q3 = matriz[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - multiplicador * IQR
    limite_superior = Q3 + multiplicador * IQR

    # Encontra índices dos outliers na coluna
    indices = matriz[(matriz[coluna] < limite_inferior) | (matriz[coluna] > limite_superior)].index
    outlier_indices.update(indices)

    # Salva a quantidade de outliers da coluna
    contagem_por_coluna[coluna] = len(indices)

# Converte o set para lista antes de indexar
outliers_iqr = matriz.loc[list(outlier_indices)]

# Impressão dos resultados
print("Quantidade de outliers por coluna (IQR):")
for coluna, qtd in contagem_por_coluna.items():
    print(f"{coluna}: {qtd}")

print(f"\nTotal de linhas com pelo menos um outlier: {len(outliers_iqr)}")

print("\nOutliers detectados com IQR (linhas):")
print(outliers_iqr)




## 6.3.  Isolation Forest

Isolation Forest

* Detecção de Outliers: O objetivo principal do código é identificar instâncias que são consideradas outliers, ou seja, pontos que estão significativamente distantes dos dados "normais". A detecção de outliers é importante porque esses valores podem distorcer análises estatísticas ou influenciar negativamente o desempenho de modelos de aprendizado de máquina.

* Isolation Forest: É uma técnica eficiente para detectar outliers, especialmente quando os dados são grandes ou de alta dimensão. Em comparação com métodos tradicionais como o Z-Score ou IQR, o Isolation Forest é mais eficaz em identificar outliers em dados que não seguem uma distribuição normal ou que são complexos.

* Análise Complementar: A comparação entre os valores dos outliers e os valores normais usando o IQR serve para verificar se os outliers identificados estão realmente fora do intervalo esperado e se esse comportamento é consistente com a definição clássica de outlier.

In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

# Seleciona apenas colunas numéricas
variaveis_numericas = matriz.select_dtypes(include='number')

# Inicializa e treina o modelo
# 0.02 -> 5% de outliers
iso_forest = IsolationForest(contamination=0.02, random_state=42)
outlier_flags = iso_forest.fit_predict(variaveis_numericas)

# Adiciona a predição ao DataFrame (-1 = outlier)
matriz['Outlier_IF'] = outlier_flags

# Total de instâncias com outliers
total_outliers = (outlier_flags == -1).sum()
print(f"Total de instâncias com outliers (Isolation Forest): {total_outliers} de {len(matriz)}")

# Análise por coluna
print("\nNúmero de outliers detectados por variável (Isolation Forest):")
col_outliers = {}
for col in variaveis_numericas.columns:
    # Compara distribuição dos valores da variável entre outliers e não-outliers
    outlier_vals = matriz.loc[matriz['Outlier_IF'] == -1, col]
    normal_vals = matriz.loc[matriz['Outlier_IF'] == 1, col]

    # Conta quantos outliers estão fora do intervalo interquartil da parte "normal"
    q1 = normal_vals.quantile(0.25)
    q3 = normal_vals.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    count = ((outlier_vals < lower) | (outlier_vals > upper)).sum()
    col_outliers[col] = count
    print(f"{col}: {count} instâncias")

# (Opcional) Ver gráfico ou salvar como DataFrame
# pd.Series(col_outliers).sort_values(ascending=False).plot(kind='barh')


## 6.4. Remoção dos outliers

O código que serve para filtrar e remover as instâncias identificadas como outliers do DataFrame, após a aplicação do modelo Isolation Forest.

In [ ]:
# Filtra apenas as linhas normais (não outliers)
matriz_sem_outliers = matriz[matriz['Outlier_IF'] == 1].drop(columns='Outlier_IF').reset_index(drop=True)

print(f"Número de instâncias após remoção de outliers: {len(matriz_sem_outliers)}")


# 7. Salvar a matriz

Salva a matriz após todo o tratamento como matriz_EDA

In [ ]:
from google.colab import files

# Salvar o DataFrame como CSV
matriz_sem_outliers.to_csv('matriz_EDA.csv', index=False)

# Fazer o download do arquivo CSV
files.download('matriz_EDA.csv')